In [2]:
import copy

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from skopt.space import Real, Categorical, Integer

import sys
sys.path.append('G:\내 드라이브\대학\대외\2023\연구원\연구과제\Ensemble-Kernel')

from Ensemble_Kernel_New import split_data,prepare_response_variable, c_o, nom, new_kernel, train_predict_fastKernelSurvivalSVM_bayesian, train_predict_fastKernelSurvivalSVM_clinical_bayesian, one_hot_encode_columns, remove_high_vif_columns, convert_categorical_to_boolean, new_kernel_no_abs, compare_kernels_with_abs_and_no_abs, train_predict_fastKernelSurvivalSVM_bayesian_no_abs, train_predict_fastKernelSurvivalSVM, train_predict_fastKernelSurvivalSVM_AFT, new_kernel_AFT, train_predict_fastKernelSurvivalSVM_bayesian_AFT, train_predict_fastKernelSurvivalSVM_clinical, train_predict_fastKernelSurvivalSVM_clinical_bayesian, C_index_fastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel

from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM, FastKernelSurvivalSVM
from sklearn.model_selection import ShuffleSplit, GridSearchCV

### 전처리

In [13]:
data = pd.read_csv("veteran_data.csv")

df = data.drop(['Unnamed: 0'], axis=1)
df = df.astype({'trt' : 'category', 'celltype' : 'category', 'prior' : 'category'})
df.replace({'celltype' : {'squamous' : 0, 'large' : 1, 'smallcell' : 2, 'adeno' : 3}}, inplace = True)

df = df.rename(columns={'age':'Age',
                        'time':'OS',
                        'status':'Status'})

#onehot encode
df_onehot, categorical_columns = one_hot_encode_columns(df)

columns_to_convert=['celltype_1', 'celltype_2','celltype_0', 'prior_10', 'trt_2']

df_onehot[columns_to_convert] = df_onehot[columns_to_convert].astype('category')

df_onehot

,OS,Status,karno,diagtime,Age,celltype_1,celltype_2,celltype_0,prior_10,trt_2
0,72,1,60,7,69,0,0,1,0,0
1,411,1,70,5,64,0,0,1,1,0
2,228,1,60,3,38,0,0,1,0,0
3,126,1,60,9,63,0,0,1,1,0
4,118,1,70,11,65,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
132,133,1,75,1,65,1,0,0,0,1
133,111,1,60,5,64,1,0,0,0,1
134,231,1,70,18,67,1,0,0,1,1
135,378,1,80,4,65,1,0,0,0,1


In [14]:
#train, test split
x_train, x_test, target_train, target_test = split_data(df_onehot)

y_train = prepare_response_variable(target_train)
y_test = prepare_response_variable(target_test)

#Linear
x_train_linear=x_train.drop(['OS','Status'],axis=1)
x_test_linear=x_test.drop(['OS','Status'],axis=1)

In [30]:
x_train

,karno,diagtime,Age,celltype_1,celltype_2,celltype_0,prior_10,trt_2,OS,Status
71,80,3,48,0,0,1,0,1,87,0
21,60,5,67,0,1,0,0,0,97,0
63,90,2,62,1,0,0,0,0,182,0
20,40,3,55,0,1,0,0,0,123,0
109,99,3,57,0,0,0,0,1,83,0
...,...,...,...,...,...,...,...,...,...,...
8,50,18,43,0,0,1,0,0,314,1
2,60,3,38,0,0,1,0,0,228,1
132,75,1,65,1,0,0,0,1,133,1
108,40,5,69,0,0,0,1,1,18,1


In [16]:
#KFold setting
cv=KFold(n_splits = 5, shuffle=True, random_state=36)

param_space = {
    'alpha': Real(1e-6, 1e+6, 'log-uniform'),
}
param_grid = {'alpha': 2. ** np.arange(-12, 13, 2)}

### Cross Validation

In [72]:
#랜덤하게 순서 섞기
x_train = x_train.sample(frac=1).reset_index(drop=True)

# train data를 그룹으로 나누기
num_groups = 3
group_size = len(x_train) // num_groups

x_groups = []

for i in range(num_groups):
    if i < num_groups - 1:
        group = x_train.iloc[i * group_size:(i + 1) * group_size]
    else:
        group = x_train.iloc[i * group_size:]
    
    x_groups.append(group)

In [84]:
cindex=[]

for i in range(len(x_groups)):
    temp = []
    for j in range(len(x_groups)):
        if i != j:
            temp.append(x_groups[j])
    #train = train data(등분한 것 중 1개를 제외한 나머지)
    train=pd.concat(temp)

    #validation = validation data(등분한 것 중 1개)
    validation = x_groups[i]

    train_target=train[['Status','OS']]
    test_target=validation[['Status','OS']]

    train_y=prepare_response_variable(train_target)
    test_y=prepare_response_variable(test_target)
    #data정의 -> train, validation, train_y, test_y

    result = C_index_fastKernelSurvivalSVM(train, validation, train_y, test_y, param_grid, keywords = ['Age', 'Sex'], drop=False, coef_drop=None)
    print(result[0])



[( True,   8) ( True, 392) ( True,  84) ( True,  53) ( True, 283)
 ( True,  61) ( True,   7) (False,  87) ( True,  99) ( True,  18)
 ( True,  12) ( True, 287) ( True, 587) ( True, 389) ( True, 110)
 ( True,  51) ( True, 216) ( True, 162) ( True,  12) ( True, 153)
 ( True, 467) ( True,  30) ( True, 126) ( True,  54) ( True,  43)
 ( True, 122) ( True, 991) (False,  83) ( True, 132) ( True, 278)
 ( True,  29) ( True,  36) ( True, 260) (False,  25) ( True, 314)
 ( True,  18) ( True,  44) ( True, 139) ( True,   8) ( True, 340)
 ( True, 133) ( True,  20) ( True,  54) ( True, 999) ( True,  45)
 ( True,   7) ( True,  51) ( True,  95) ( True,  22) ( True, 140)
 ( True,  51) ( True,  95) ( True, 228) ( True,   2) ( True,  52)
 ( True,  82) ( True, 118) ( True, 250) ( True,  25) ( True, 186)
 (False,  97)]
[( True,  80) ( True, 111) ( True,  59) ( True,  73) ( True,  10)
 ( True, 242) ( True,  24) ( True,  92) ( True,  72) (False, 123)
 ( True, 111) ( True,  11) ( True,  24) ( True,  13) ( True, 

In [ ]:
C_index_fastKernelSurvivalSVM(x_groups[0], y_groups[0], x_test, y_test, param_grid, keywords = ['Age', 'Sex'], drop=False, coef_drop=None)

In [ ]:
'''
results=pd.DataFrame()
results['train_C_index']=[]
results['test_C_index']=[]
results['remaining_variables']=[]

for i in range(0,100):

    x_train, x_test, target_train, target_test = split_data(df_onehot, randomState=random_state[i])

    y_train = prepare_response_variable(target_train)
    y_test = prepare_response_variable(target_test)

    # Create kernel using x_train
    train_kernel, coef, coef_drop, remaining_variables = new_kernel(x_train, drop=False, keywords = ['Age','Sex'])

    # Grid Optimization
    kssvm = FastKernelSurvivalSVM(optimizer = "rbtree", kernel = 'precomputed', max_iter = 1000, tol = 1e-6, random_state=36)
    kgcv = GridSearchCV(kssvm, param_grid, n_jobs=-1, refit=True, cv=KFold(n_splits = 5, shuffle=True, random_state=36))

    # Fit model
    kgcv.fit(train_kernel, y_train)

    # Optimal hyperparameters and c_index
    best_params = kgcv.best_params_
    best_c_index = kgcv.best_score_

    # Predict on x_train and calculate c_index
    train_pred = kgcv.predict(train_kernel)
    train_c_index = concordance_index_censored(y_train["Status"], y_train["OS"], train_pred)

    # Predict on x_test and calculate c_index
    test_kernel, _, _, _ = new_kernel(x_train, x_test, coef=coef, drop=False, coef_drop=coef_drop, keywords=['Age', 'Sex'])
    test_pred = kgcv.predict(test_kernel)
    test_c_index = concordance_index_censored(y_test["Status"], y_test["OS"], test_pred)

    results=results.append({"train_C_index":train_c_index[0],"test_C_index":test_c_index[0],"remaining_variables":remaining_variables},
                            ignore_index=True)
        
results.to_csv("veteran_cox.csv",index=False,encoding='cp949')
'''

In [17]:
#random state 추출
import random

with open("random_state_100.txt", "r") as file:
    random_state=file.read()
    
random_state=random_state.split("\n")
random_state=[int(x) for x in random_state if x]

In [ ]:
x_train, x_test, target_train, target_test = split_data(df_onehot, randomState=42)
y_train = prepare_response_variable(target_train)
y_test = prepare_response_variable(target_test)

results=pd.DataFrame()
results['train_C_index']=[]
results['test_C_index']=[]
results['remaining_variables']=[]

# Create kernel using x_train
train_kernel, coef, coef_drop, remaining_variables = new_kernel(x_train, drop=False, keywords = ['Age','Sex'])

# Grid Optimization
kssvm = FastKernelSurvivalSVM(optimizer = "rbtree", kernel = 'precomputed', max_iter = 1000, tol = 1e-6, random_state=36)
kgcv = GridSearchCV(kssvm, param_grid, n_jobs=-1, refit=True, cv=KFold(n_splits = 5, shuffle=True, random_state=36))

# Fit model
kgcv.fit(train_kernel, y_train)

# Optimal hyperparameters and c_index
best_params = kgcv.best_params_
best_c_index = kgcv.best_score_

# Predict on x_train and calculate c_index
train_pred = kgcv.predict(train_kernel)
train_c_index = concordance_index_censored(y_train["Status"], y_train["OS"], train_pred)

# Predict on x_test and calculate c_index
test_kernel, _, _, _ = new_kernel(x_train, x_test, coef=coef, drop=False, coef_drop=coef_drop, keywords=['Age', 'Sex'])
test_pred = kgcv.predict(test_kernel)
test_c_index = concordance_index_censored(y_test["Status"], y_test["OS"], test_pred)

results=results.append({"train_C_index":train_c_index[0],"test_C_index":test_c_index[0],"remaining_variables":remaining_variables},
                            ignore_index=True)